In [1]:
import requests
import pandas as pd
from IPython.display import display

def fetch_mobula_data():
    """Fetch cryptocurrency data from Mobula API with error handling"""
    url = "https://production-api.mobula.io/api/1/all"
    querystring = {
        "fields": "logo,price,price_change_1h,price_change_24h,price_change_7d,"
                  "price_change_1y,market_cap,liquidity,"
                  "blockchains,chat,twitter,website"
    }
    
    try:
        response = requests.get(url, params=querystring, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"API Request Failed: {e}")
        return None

def process_crypto_data(raw_data):
    """Process raw API response into structured DataFrame"""
    if not raw_data or 'data' not in raw_data:
        return pd.DataFrame()
    
    processed = []
    for asset in raw_data['data']:
        # Handle nested structures
        contracts = asset.get('contracts', {})
        social = asset.get('chat', {})
        
        # Extract core metrics
        asset_data = {
            'id': asset.get('id'),
            'name': asset.get('name'),
            'symbol': asset.get('symbol'),
            'price': asset.get('price'),
            'market_cap': asset.get('market_cap'),
            'liquidity': asset.get('liquidity'),
            'blockchains': ', '.join(asset.get('blockchains', [])) or None,
            'price_change_1h': asset.get('price_change_1h'),
            'price_change_24h': asset.get('price_change_24h'),
            'price_change_7d': asset.get('price_change_7d'),
            'price_change_1y': asset.get('price_change_1y'),
            'twitter': asset.get('twitter'),
            'website': asset.get('website'),
            'logo': asset.get('logo'),
        }
        
        
        # Extract social links (assuming chat remains a dictionary)
        if isinstance(social, dict):
            for platform, link in social.items():
                asset_data[f'social_{platform.lower()}'] = link
        elif isinstance(social, list):
            # In case social data comes as a list, you can join them into a string.
            asset_data['social'] = ', '.join(social)
        
        processed.append(asset_data)
    
    return pd.DataFrame(processed)

# Execute data pipeline
raw_data = fetch_mobula_data()
if raw_data:
    df = process_crypto_data(raw_data)
    
    # Clean numerical fields
    numeric_cols = ['price', 'market_cap', 'liquidity', 
                    'price_change_1h', 'price_change_24h',
                    'price_change_7d',  'price_change_1y']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    
    # Display processed data
    print(f"Found {len(df)} cryptocurrencies")
    display(df.head(3))
    
    # Save to CSV
    df.to_csv('crypto_assets.csv', index=False)
else:
    print("No data processed")


C:\Users\mahmo\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Found 26830 cryptocurrencies


,id,name,symbol,price,market_cap,liquidity,blockchains,price_change_1h,price_change_24h,price_change_7d,price_change_1y,twitter,website,logo
0,2,R1C4RD0,ALGO,0.100000,1.499471e+07,1359.012869,Arbitrum,0.000000,0.000000,0.000000,-1.168324,https://twitter.com/CapitalLys,https://lyscapital.io/,https://metacore.mobula.io/fde8226ca6b18057c48...
1,3,StaySAFU,SAFU,2.384893,3.973470e+04,26378.360656,BNB Smart Chain (BEP20),-4.833419,-4.833419,-7.429474,4.398327,https://twitter.com/StaySAFUOrg,https://staysafu.org/,https://coin-images.coingecko.com/coins/images...
2,4,Fold,FLD,0.001228,1.367012e+06,137719.635011,Avalanche C-Chain,-0.088982,9.892164,-2.307241,-80.851764,https://twitter.com/LabNinety1,https://www.ninety1.io/,https://coin-images.coingecko.com/coins/images...


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26830 entries, 0 to 26829
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                26830 non-null  int64  
 1   name              26830 non-null  object 
 2   symbol            26830 non-null  object 
 3   price             26485 non-null  float64
 4   market_cap        26830 non-null  float64
 5   liquidity         26830 non-null  float64
 6   blockchains       17400 non-null  object 
 7   price_change_1h   26830 non-null  float64
 8   price_change_24h  26830 non-null  float64
 9   price_change_7d   26830 non-null  float64
 10  price_change_1y   26830 non-null  float64
 11  twitter           26600 non-null  object 
 12  website           26726 non-null  object 
 13  logo              26678 non-null  object 
dtypes: float64(7), int64(1), object(6)
memory usage: 2.9+ MB


In [3]:
df.describe()

,id,price,market_cap,liquidity,price_change_1h,price_change_24h,price_change_7d,price_change_1y
count,2.683000e+04,2.648500e+04,2.683000e+04,2.683000e+04,2.683000e+04,2.683000e+04,2.683000e+04,2.683000e+04
mean,6.327054e+07,4.231874e+13,2.447215e+08,1.849955e+05,1.320692e+36,1.320698e+36,1.223237e+37,1.310134e+36
std,4.943730e+07,4.894763e+15,2.511582e+10,4.783840e+06,2.163276e+38,2.163276e+38,1.799531e+39,2.137591e+38
min,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+02,-1.000000e+02,-1.000000e+02,-1.000000e+02
25%,7.317250e+03,6.407834e-09,0.000000e+00,0.000000e+00,-1.334631e-01,-6.917522e+00,-1.719549e+01,-9.235572e+01
50%,1.000067e+08,8.778884e-05,1.666500e+03,1.286491e+03,0.000000e+00,-1.510757e+00,-7.800987e+00,-5.846732e+01
75%,1.024842e+08,6.681791e-03,1.484928e+05,1.589157e+04,3.668381e-01,0.000000e+00,0.000000e+00,0.000000e+00
max,1.025041e+08,6.172869e+17,3.787086e+12,5.059230e+08,3.543418e+40,3.543418e+40,2.926244e+41,3.501320e+40


In [4]:
df.isnull().sum()

id                     0
name                   0
symbol                 0
price                345
market_cap             0
liquidity              0
blockchains         9430
price_change_1h        0
price_change_24h       0
price_change_7d        0
price_change_1y        0
twitter              230
website              104
logo                 152
dtype: int64

In [17]:
nan_price_symbols = df[df['price'].isna()]['symbol']
display(nan_price_symbols)

20932      RSETH
21226     USDC.E
21303      SMBTC
21308        BVR
21315    XSTABLE
          ...   
26459      DCOIN
26462      AIGPT
26468      AWETH
26469      AUSDC
26475       FORM
Name: symbol, Length: 334, dtype: object

In [18]:
df.isnull().sum()

id                     0
name                   0
symbol                 0
price                334
market_cap             0
liquidity              0
blockchains         9420
price_change_1h        0
price_change_24h       0
price_change_7d        0
price_change_1y        0
twitter              735
website              634
logo                 151
dtype: int64

In [19]:
df = df.dropna(subset=['price'])

In [20]:
df.isnull().sum()

id                     0
name                   0
symbol                 0
price                  0
market_cap             0
liquidity              0
blockchains         9279
price_change_1h        0
price_change_24h       0
price_change_7d        0
price_change_1y        0
twitter              701
website              600
logo                 151
dtype: int64

In [21]:
# Fill NaN values with 'Unknown' for specified columns
df['blockchains'].fillna('Unknown', inplace=True)
df['twitter'].fillna('Unknown', inplace=True)
df['website'].fillna('Unknown', inplace=True)

# Set a default image link for NaN values in the 'logo' column
default_logo_url = 'No Logo Available'
df['logo'].fillna(default_logo_url, inplace=True)

# Display the updated DataFrame
display(df.head(3))

C:\Users\mahmo\AppData\Local\Temp\ipykernel_29288\391515113.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['blockchains'].fillna('Unknown', inplace=True)
C:\Users\mahmo\AppData\Local\Temp\ipykernel_29288\391515113.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

,id,name,symbol,price,market_cap,liquidity,blockchains,price_change_1h,price_change_24h,price_change_7d,price_change_1y,twitter,website,logo
0,2,R1C4RD0,ALGO,0.100000,1.499471e+07,1359.012869,Arbitrum,0.000000,0.000000,0.000000,-1.168324,https://twitter.com/CapitalLys,https://lyscapital.io/,https://metacore.mobula.io/fde8226ca6b18057c48...
1,3,StaySAFU,SAFU,2.828294,4.712221e+04,31143.732710,BNB Smart Chain (BEP20),-1.384072,-2.502263,-3.784805,53.319801,https://twitter.com/StaySAFUOrg,https://staysafu.org/,https://coin-images.coingecko.com/coins/images...
2,4,Fold,FLD,0.001509,1.675527e+06,165397.606568,Avalanche C-Chain,-10.741872,-22.350238,-27.593057,-52.244697,https://twitter.com/LabNinety1,https://www.ninety1.io/,https://coin-images.coingecko.com/coins/images...


In [22]:
df.isnull().sum()

id                  0
name                0
symbol              0
price               0
market_cap          0
liquidity           0
blockchains         0
price_change_1h     0
price_change_24h    0
price_change_7d     0
price_change_1y     0
twitter             0
website             0
logo                0
dtype: int64

In [23]:
df.to_csv('crypto_assets.csv', index=False)